In [ ]:
from torch.utils import data as data
import pandas as pd
from torchvision.transforms.functional import normalize
from tqdm import tqdm
import os
from basicsr.data.data_util import (paired_paths_from_folder,
                                    paired_paths_from_lmdb,
                                    paired_paths_from_meta_info_file)
from basicsr.data.transforms import augment, paired_random_crop
from basicsr.utils import FileClient, imfrombytes, img2tensor, padding
from torch.utils.data.dataloader import default_collate
import h5py
# local modules
from basicsr.data.h5_augment import *

from torch.utils.data import ConcatDataset
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os

def transform_voxel(voxel, transpose_to_CHW=False, norm_voxel = True):
    """
    Augment voxel and turn into tensor
    @param voxel Input voxel
    @param seed  Seed for random number generation
    @returns Augmented voxel
    """

    if transpose_to_CHW:
        voxel = torch.from_numpy(voxel.transpose(2, 0, 1)).float()# H,W,C -> C,H,W

    else:
        if norm_voxel:
            voxel = torch.from_numpy(voxel).float() / abs(max(voxel.min(), voxel.max(), key=abs))  # -1 ~ 1
        else:
            voxel = torch.from_numpy(voxel).float()

    # if self.vox_transform:
    #     random.seed(seed)
    #     voxel = self.vox_transform(voxel)
    return voxel

In [1]:
import os
import h5py
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Data path
data_path = '/workspace/data/GOPRO/train/'
h5_folder = sorted(os.listdir(data_path))

output_mother_path = '/workspace/FFTformer/results/Gopro_train/visualization'

# Initialize KL divergence sums
kl_div_refined = 0
kl_div_event = 0
kl_div_another_event = 0
count = 0  # 샘플 개수 카운트

# Scaling factor for Softmax conversion
scale_factor = 5.0

# Iterate through H5 files
for h5 in tqdm(h5_folder, ncols=80):
    imgs_path = os.path.join(data_path, h5)
    scene = h5[:-3]
    scene_folder = sorted(os.listdir(os.path.join(output_mother_path, scene)))

    with h5py.File(imgs_path, 'r') as h5_file:
        refined_events = h5_file['gen_event_refined']
        voxels = h5_file['voxels']
        events = h5_file['gen_event']

        for i, event in enumerate(refined_events):
            # Load datasets
            refined_event = np.array(h5_file['gen_event_refined'][event])
            voxel = np.array(h5_file['voxels']['voxel{:09d}'.format(i)])  # GT
            event = np.array(h5_file['gen_event']['image{:09d}'.format(i)])
            another_event = np.load(os.path.join(output_mother_path, scene, scene_folder[i], 'out.npy'))

            # Convert to torch tensors
            refined_event = torch.tensor(refined_event, dtype=torch.float32)
            voxel = torch.tensor(voxel, dtype=torch.float32)  # GT
            event = torch.tensor(event, dtype=torch.float32)
            another_event = torch.tensor(another_event, dtype=torch.float32)

            # Apply Softmax after scaling
            refined_event_prob = F.softmax(refined_event * scale_factor, dim=-1) + 1e-8
            voxel_prob = F.softmax(voxel * scale_factor, dim=-1) + 1e-8  # GT
            event_prob = F.softmax(event * scale_factor, dim=-1) + 1e-8
            another_event_prob = F.softmax(another_event * scale_factor, dim=-1) + 1e-8

            # Compute KL Divergence
            kl_div_refined += F.kl_div(refined_event_prob.log(), voxel_prob, reduction='batchmean').item()
            kl_div_event += F.kl_div(event_prob.log(), voxel_prob, reduction='batchmean').item()
            kl_div_another_event += F.kl_div(another_event_prob.log(), voxel_prob, reduction='batchmean').item()

            count += 1  # 샘플 개수 카운트
    break
# 평균 KL Divergence 계산
kl_div_refined /= count
kl_div_event /= count
kl_div_another_event /= count

# Print results
print("\nKL Divergence Summary (Compared to GT: Voxel):")
print(f"Refined Event KL Divergence: {kl_div_refined:.6f}")
print(f"Event KL Divergence: {kl_div_event:.6f}")
print(f"Another Event KL Divergence: {kl_div_another_event:.6f}")


/opt/conda/envs/deblur/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|                                                    | 0/22 [00:10<?, ?it/s]


KL Divergence Summary (Compared to GT: Voxel):
Refined Event KL Divergence: 4049.223441
Event KL Divergence: 4368.650849
Another Event KL Divergence: 4166.718810


In [14]:
import torch.nn.functional as F
import torch

# 리스트를 Tensor로 변환
tensor_input = torch.tensor([-5,3.5,5], dtype=torch.float32)

# Softmax 적용
result = F.softmax(tensor_input, dim=-1)
print(result)


tensor([3.7116e-05, 1.8242e-01, 8.1754e-01])
